## Hash Sets and Hash Maps
- Hash sets and [hash maps](https://en.wikipedia.org/wiki/Hash_table) are data structures commonly used in programming. They are a bit similar to (named-) lists, but they can perform certain operations much faster.
- Implementation of hash sets/maps is unfortunatelly not directly available in the base R, but they can be used via the package [r2r](https://cran.r-project.org/web/packages/r2r/index.html) (under the hood, the package rellies on the usage of R [environments](https://adv-r.hadley.nz/environments.html)).
- Hash sets behaves the same way as [sets](https://en.wikipedia.org/wiki/Set_(mathematics)) in the mathematical sense. This differ from the way how lists operate:
    - Each element can be included in the set at most once.
    - Sets are not ordered.
- Hash maps works similarly as named lists in R, but the keys (names) can be arbitrary objects instead of just strings. Also, the time complexity of using a key to access a value is a constant instead of a linear time.

In [1]:
#install.packages('r2r')
#install.packages('microbenchmark')
options(repr.plot.width = 16, repr.plot.height = 10)
library(r2r)
library(microbenchmark)

### Hashing allows access in constant time
- Starting with R lists and vectors: the good thing is that they allow constant time $\mathcal{O}(1)$ access to their elements, if we use the integer index.

In [2]:
small_list = as.list(1:10) # The elements of the list can be anything, we just use numbers here for simplicity
very_large_list = as.list(1:10^6)

bench_read_1 <- microbenchmark(x <- small_list[[7]])
bench_read_2 <- microbenchmark(x <- very_large_list[[456789]])

print(bench_read_1)
print(bench_read_2)

Unit: nanoseconds
                 expr min lq   mean median uq  max neval
 x <- small_list[[7]]  95 97 146.66     98 99 4247   100
Unit: nanoseconds
                           expr min  lq   mean median  uq  max neval
 x <- very_large_list[[456789]]  98 100 131.03    101 102 2901   100


- So far so good: accessing elements in the lists took about same time for both small and large list. Also, the operation is super fast in general, as it took only about 150 nanoseconds.
- However, things get worse if we want to check for presence of certain object (number, string...) in the list, using the **%in%** operator. The R simply tries to compare the object to every element in the list. If the list get larger, then there will be (in the worst case, or on average) more comparisons needed to be made. This is a linear time $\mathcal{O}(n)$ operation.

In [3]:
list_sizes <- 10^(1:6)
results <- data.frame(n = character(), time = numeric())

for (n in list_sizes) {    
    tested_list <- as.list(sample(1:n, n, replace = TRUE))    
    bench_read <- microbenchmark(is_included <- sample(1:n, 1) %in% tested_list, unit = "ns", times=20)     
    results <- rbind(results, data.frame(n = as.character(n), time = bench_read$time))
}

ggplot(results, aes(x = n, y = time)) +
    geom_boxplot() +
    scale_y_log10() +
    labs(x = "List size (n)", y = "Runtime (ns)", title = "List Membership Test Performance") +
    theme_minimal() +
    theme(axis.title = element_text(size = 18),
        axis.text = element_text(size = 11),
        plot.title = element_text(size = 20, face = "bold", hjust = 0.5))


ERROR: Error in ggplot(results, aes(x = n, y = time)): could not find function "ggplot"


Doing the same using hashsets:

In [ ]:
library(microbenchmark)
library(ggplot2)

set_sizes <- 10^(1:6)
results <- data.frame(n = character(), time = numeric())

for (n in set_sizes) {
    tested_hashset <- do.call(hashset, as.list(sample(1:n, n))) 
    bench_read <- microbenchmark(is_included <- tested_hashset[[sample(1:n, 1)]], unit = "ns", times=100)     
    results <- rbind(results, data.frame(n = as.character(n), time = bench_read$time))
}

ggplot(results, aes(x = n, y = time)) +
    geom_boxplot() +
    scale_y_log10() +
    labs(x = "Hashset size", y = "Runtime (ns)", title = "Hash Membership Test Performance") +
    theme_minimal() +
    theme(axis.title = element_text(size = 18),
        axis.text = element_text(size = 11),
        plot.title = element_text(size = 20, face = "bold", hjust = 0.5))

- The [hashing](https://en.wikipedia.org/wiki/Hash_function) works the same way as accessing a list by integer index - the hash of the object specify a place in memory where the object should be located. R can go directly to that place, without iterating over all elements in the set. Accesing the element in hash set is therefore constant time $\mathcal{O}(1)$ operation.